In [ ]:
!pip install pythainlp

Use Top 10 by TF-IDF Score ranking

In [ ]:
import pandas as pd
from pythainlp.util import normalize
from pythainlp.tokenize import word_tokenize

# Load your CSV file
projects_df = pd.read_csv('/content/pasi_61_65_dataset.csv')  # Replace 'projects.csv' with your actual file name

# Sample keywords DataFrame with multiple keywords per strategy label
keywords_df = pd.DataFrame({
    'strategy_label': [
        'ประเด็นการพัฒนาที่ 1 : พัฒนาสภาพแวดล้อมเมืองให้น่าอยู่ เอื้อต่อการขยายตัวทางเศรษฐกิจและสังคม',
        'ประเด็นการพัฒนาที่ 2 : ยกระดับการผลิตอาหาร สินค้าเกษตร อุตสาหกรรม ให้มีคุณภาพและได้มาตรฐาน สู่ตลาดโลก',
        'ประเด็นการพัฒนาที่ 3 : พัฒนามาตรฐานการท่องเที่ยว เชื่อมโยงธุรกิจและบริการด้านการท่องเที่ยว',
        'ประเด็นการพัฒนาที่ 4 : พัฒนาและเสริมสร้างศักยภาพคนและชุมชนให้มีคุณภาพเพื่อรองรับการเปลี่ยนแปลงโครงสร้างประชากรและสังคม'
    ],
    'keywords': [
        ['เขื่อน','ฝาย','ลำน้ำ','ลำห้วย','ห้วย',
         'ตลิ่ง',
         'ภูมิทัศน์',
         'น้ำ',
         'ทางหลวง','ช่องจราจร'
         'แม่น้ำ','คลอง',
         'ชุมชน','หมู่บ้าน'
         'ขยะ','ขยะมูลฝอย'],

        ['สินค้า','ตลาด',
         'เกษตร','เกษตรกร',
         'อาหาร','เนื้อ','ปศุสัตว์',
         'ผลิตภัณฑ์',
         'อุตสาหกรรม',
         'สากล'],

        ['วิถี',
         'ชุมชน', 'หมู่บ้าน',
         'วัฒนธรรม',
         'ประเพณี',
         'แหล่งท่องเที่ยว','สถานที่สำคัญ','สถานที่ท่องเที่ยว','นักท่องเที่ยว'
         'แรลลี่',
         'ป้าย','แผ่นป้าย'
         'สื่อ'],

        ['คอนกรีตเสริมเหล็ก','คอนกรีต'
         'ถนน','ซอย','สะพาน'
         'ท่อ',
         'แรงดัน',
         'บ่อพัก',
         'บ่อ',
         'ผิวจราจร',
         'แรงงาน',
         'ผู้สูงอายุ',
         ]
    ]
})

# Normalize and tokenize Thai text
def process_text(text):
    normalized_text = normalize(str(text))  # Ensure text is string
    return word_tokenize(normalized_text, keep_whitespace=False)

# Match keywords and assign only the first matching strategy label
def assign_strategy(project_name_tokens, keywords_df):
    for _, row in keywords_df.iterrows():
        strategy_label = row['strategy_label']
        keywords = row['keywords']
        # Tokenize and normalize all keywords
        keyword_tokens_list = [process_text(keyword) for keyword in keywords]
        # Flatten the tokenized keywords into a single list
        keyword_tokens = [token for tokens in keyword_tokens_list for token in tokens]
        # Check if any keyword matches the project name tokens
        if any(token in project_name_tokens for token in keyword_tokens):
            return strategy_label  # Return the first matching strategy label
    return None  # No matching label

# Process the 'project_name' column in your CSV
projects_df['project_name_tokens'] = projects_df['project_name'].apply(process_text)
projects_df['strategy_label'] = projects_df['project_name_tokens'].apply(
    lambda tokens: assign_strategy(tokens, keywords_df)
)

# Drop intermediate tokens column if not needed
projects_df.drop(columns=['project_name_tokens'], inplace=True)

# Save the updated DataFrame to a new CSV
projects_df.to_csv('projects_61_65_with_strategy.csv', index=False)

print("Processed data saved to 'projects_61_65_with_strategy.csv'")


Processed data saved to 'projects_61_65_with_strategy.csv'


Use LLm intersections word

In [ ]:
import pandas as pd
from pythainlp.util import normalize
from pythainlp.tokenize import word_tokenize

# Load your CSV file
projects_df = pd.read_csv('/content/pasi_61_65_dataset.csv')  # Replace 'projects.csv' with your actual file name

# Sample keywords DataFrame with multiple keywords per strategy label
keywords_df = pd.DataFrame({
    'strategy_label': [
        'ประเด็นการพัฒนาที่ 1 : พัฒนาสภาพแวดล้อมเมืองให้น่าอยู่ เอื้อต่อการขยายตัวทางเศรษฐกิจและสังคม',
        'ประเด็นการพัฒนาที่ 2 : ยกระดับการผลิตอาหาร สินค้าเกษตร อุตสาหกรรม ให้มีคุณภาพและได้มาตรฐาน สู่ตลาดโลก',
        'ประเด็นการพัฒนาที่ 3 : พัฒนามาตรฐานการท่องเที่ยว เชื่อมโยงธุรกิจและบริการด้านการท่องเที่ยว',
        'ประเด็นการพัฒนาที่ 4 : พัฒนาและเสริมสร้างศักยภาพคนและชุมชนให้มีคุณภาพเพื่อรองรับการเปลี่ยนแปลงโครงสร้างประชากรและสังคม'
    ],

    'keywords': [
        ["สิ่งแวดล้อม","ทรัพยากรธรรมชาติ","ระบบนิเวศ","การอนุรักษ์","การพัฒนา",
         "ชุมชน", "หมู่บ้าน","ชนบท","ที่อยู่อาศัย",
         "ทางหลวง","ช่องจราจร",
         "น้ำ","แหล่งน้ำ",
         "ไฟฟ้า","กระแสไฟฟ้า","พลังงานไฟฟ้า",
         "ขยะมูลฝอย","ขยะ","น้ำเสีย","สิ่งปฏิกูล","มูลฝอย"
        ],

        ["สินค้า","ตลาด","ผู้บริโภค",
         "อาหาร","บริโภค",
         "เกษตร","การเกษตร","เกษตรกรรม","เกษตรกร",
         "ผลิตภัณฑ์","วัตถุดิบ","อุตสาหกรรม",
         "ฟาร์ม","ปศุสัตว์",
         "มาตรฐานสากล","มาตรฐาน"
        ],

        ["ผลิตภัณฑ์",
         "แหล่งท่องเที่ยว","สถานที่ท่องเที่ยว","นักท่องเที่ยว","สถานที่สำคัญ",
         "อาหาร","บริโภค",
         "เทศกาล","งานเทศกาล",
         "วัฒนธรรม","ภูมิปัญญา","ศิลปวัฒนธรรม","วัฒนธรรมประเพณี",
         "ประเพณี","พิธีกรรม","ขนบธรรมเนียม","ธรรมเนียม","ขนบธรรมเนียมประเพณี",
         "ชุมชน","หมู่บ้าน","ชนบท","ที่อยู่อาศัย",
         "ภูมิทัศน์","ทิวทัศน์",
         "วิถี"
        ],

        ["ประชาชน","พลเมือง",
         "คุณภาพชีวิต","เสริมสร้าง","การพัฒนา","การส่งเสริม",
         "เยาวชน","ยุวชน","เด็ก",
         "อาชีพ",
         "ชุมชน","หมู่บ้าน","ชนบท","ที่อยู่อาศัย",
         "อาสาสมัคร","เจ้าหน้าที่","อาสา",
         "ภูมิปัญญาท้องถิ่น","ภูมิปัญญา","ศิลปวัฒนธรรม","วัฒนธรรมประเพณี",
         "ผู้สูงอายุ",
         "ทักษะ","ความสามารถ","ประสบการณ์","ความชำนาญ"
        ]

    ]
})

# Normalize and tokenize Thai text
def process_text(text):
    normalized_text = normalize(str(text))  # Ensure text is string
    return word_tokenize(normalized_text, keep_whitespace=False)

# Match keywords and assign only the first matching strategy label
def assign_strategy(project_name_tokens, keywords_df):
    for _, row in keywords_df.iterrows():
        strategy_label = row['strategy_label']
        keywords = row['keywords']
        # Tokenize and normalize all keywords
        keyword_tokens_list = [process_text(keyword) for keyword in keywords]
        # Flatten the tokenized keywords into a single list
        keyword_tokens = [token for tokens in keyword_tokens_list for token in tokens]
        # Check if any keyword matches the project name tokens
        if any(token in project_name_tokens for token in keyword_tokens):
            return strategy_label  # Return the first matching strategy label
    return None  # No matching label

# Process the 'project_name' column in your CSV
projects_df['project_name_tokens'] = projects_df['project_name'].apply(process_text)
projects_df['strategy_label'] = projects_df['project_name_tokens'].apply(
    lambda tokens: assign_strategy(tokens, keywords_df)
)

# Drop intermediate tokens column if not needed
projects_df.drop(columns=['project_name_tokens'], inplace=True)

# Save the updated DataFrame to a new CSV
projects_df.to_csv('projects_61_65_LLM_with_strategy.csv', index=False)

print("Processed data saved to 'projects_61_65_LLM_with_strategy.csv'")

Processed data saved to 'projects_61_65_LLM_with_strategy.csv'
